# Course 3: Cleaning Data with PySpark

In [1]:
from pyspark.sql import SparkSession

# Create SparkSession from builder
# If the sample data you work with is small, you can remove the `.config` call
spark = SparkSession.builder.appName('Spark').config("spark.driver.memory", "15g").getOrCreate()

23/03/31 10:15:07 WARN Utils: Your hostname, Mufins-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.21 instead (on interface en0)
23/03/31 10:15:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/31 10:15:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Get SparkContext
sc = spark.sparkContext

## 1. Introduction to Data Cleaning with Apache Spark

### Data Cleaning with Spark

Data cleaning is preparing raw data for use in data processing pipelines.

Possible tasks in data cleaning includes:
- Reformatting or replacing text
- Perform calculations
- Removing garbage or incomplete data

Most data cleaning systems have 2 big problems: optimizing performance and organizing the flow of data. Typical programming languages wouldn't be able to process a massive amount of information in a timely manner. Spark lets you scale your data processing capacity as your requirements evolve.

Using Spark transformations (refers to Course 2), we can create a PySpark DataFrame and continue processing afterwards.

### Spark Schema

A primary function of data cleaning is to verify all data is in the expected format. Spark provides a built-in ability to validate datasets with schemas.

A schema defines and validates the number and types of columns for a given DataFrame. A schema can contain many different types of fields (int, float, date, str, array, struct). A defined schema allows Spark to filter out data that doesn't conform during read, ensuring expected correctness.

Normally a data import will try to infer a schema on read - this requires reading the data twice. Defining a schema limits this to a single read operation.

In [3]:
# Example Spark schema
from pyspark.sql.types import *

peopleSchema = StructType([
    StructField('name', StringType(), True),
    StructField('age', IntegerType(), True),
    StructField('city', StringType(), True)
])
peopleSchema

StructType([StructField('name', StringType(), True), StructField('age', IntegerType(), True), StructField('city', StringType(), True)])

## 2. Immutability and lazy processing

Python variables are mutable and flexible. Hence, they are potential for issues with concurrency, and likely add complexity.

Unlike typical Python variables, Spark DataFrames are immutable. While not strictly required, immutability is often a component of functional programming. We won't go into everything that implies here, but understand that Spark is designed to use immutable objects. Practically, this means Spark DataFrames are defined once and are not modifiable after initialization. If the variable name is reused, the original data is removed and the variable name is reassigned to the new data. While this seems inefficient, it actually allows Spark to share data between all cluster components. It can do so without worry about concurrent data objects.



In [4]:
import pyspark.sql.functions as F

# Load the CSV file
aa_dfw_df = spark.read.format('csv').options(Header=True).load('datasets/AA_DFW_2014_Departures_Short.csv.gz')

# Add the airport column using the F.lower() method
aa_dfw_df = aa_dfw_df.withColumn('airport', F.lower(aa_dfw_df['Destination Airport']))

# Drop the Destination Airport column
aa_dfw_df = aa_dfw_df.drop(aa_dfw_df['Destination Airport'])

# Show the DataFrame
aa_dfw_df.show()

+-----------------+-------------+-----------------------------+-------+
|Date (MM/DD/YYYY)|Flight Number|Actual elapsed time (Minutes)|airport|
+-----------------+-------------+-----------------------------+-------+
|       01/01/2014|         0005|                          519|    hnl|
|       01/01/2014|         0007|                          505|    ogg|
|       01/01/2014|         0035|                          174|    slc|
|       01/01/2014|         0043|                          153|    dtw|
|       01/01/2014|         0052|                          137|    pit|
|       01/01/2014|         0058|                          174|    san|
|       01/01/2014|         0060|                          155|    mia|
|       01/01/2014|         0064|                          185|    jfk|
|       01/01/2014|         0090|                          126|    ord|
|       01/01/2014|         0096|                           91|    stl|
|       01/01/2014|         0099|                          182| 

In [5]:
aa_dfw_df.printSchema()

root
 |-- Date (MM/DD/YYYY): string (nullable = true)
 |-- Flight Number: string (nullable = true)
 |-- Actual elapsed time (Minutes): string (nullable = true)
 |-- airport: string (nullable = true)



## 3. Understanding Parquet

### Diffculties with CSV files

- No defined schema
- Nested data requires special handling
- Encoding format limited

In specific with Spark, CSV files are:

- Slow to be parsed.
- Files cannot be filtered (no predicate pushdown), reduces the amount of information that must be processed in large datasets.
- Any intermediate use required redefining schema.

### The Parquet format

Parquet is a compressed columnar data format developed for use in any Hadoop-based system (Spark, Hadoop, Apache Impala, etc.). The Parquet format is structured with data accessible in chunks, allowing efficient read/write operations without processing the entire file. This structured format supports Spark's predicate pushdown functionality, providing significant performance improvement. Finally, Parquet files automatically include schema information and handle data encoding. This is perfect for intermediary or on-disk representation of processed data. Not that Parquet files are a binary file format and can only be used with the proper tools.

Reading a parquet file into a Dataframe can be used via `spark.read.format('parquet').load(<parquet_file>)` or `spark.read.parquet(<parquet_file>)`.

Writing can be used using either `df.write.format('parquet').save(<parquet_file>)` or `df.write.parquet(<parquet_file>)`.

The long-form versions of each permit extra option flags, such as when overwriting an existing parquet file.

### Parquet and SQL

Once a dataframe is retrieved, we can use the `.createOrReplaceTempView()` then use normal SQL syntax and the `spark.sql` method (as demonstrated in Course 2).

In [6]:
# View the row count of aa_dfw_df
print(aa_dfw_df.count())

# Save the DataFrame in Parquet format
aa_dfw_df.write.format('parquet').save('datasets/example.parquet', mode='overwrite')

# Read the Parquet file into a new DataFrame and run a count
print(spark.read.parquet('datasets/example.parquet').count())

157198


157198


## 4. Dataframe operations

### Filtering data

Can be used within `df.filter()` or `df.where()`.

Some example:
- `voter_df.filter(voter_df['name'].isNotNull())`
- `voter_df.filter(voter_df.date.year > 1800)`
- `voter_df.filter(voter_df.name.like('M%'))`
- `voter_df.where(voter_df['_c0'].contains('VOTE'))`
- `voter_df.where(~ voter_df._c1.isNull())`

### String transformation

- `voter_df.withColumn('upper', F.upper('name'))`
- `voter_df.withColumn('splits', F.split('name', ' '))`
- `voter_df.withColumn('year', voter_df['_c4'].cast(IntegerType()))`

### ArrayType() column functions

- `.size(<column>)`
- `.getItem(<index>)`

### Conditional clauses

- `df.select(df.Name, df.Age, F.when(df.Age >= 60, "Elder").when(df.Age >= 18, "Adult").otherwise("Minor"))`

### User defined function

First, define a Python method:

```python
def reverseString(mystr):
    return mystr[::-1]
```

Then, wrap the function and store as a variable

```python
from pyspark.sql.functions import udf

udfReverseString = udf(reverseString, StringType())
```

Finally, use with Spark:

```python
user_df = user_df.withColumn('ReverseName', udfReverseString(user_df.Name))
```

## 5. Caching

### Caching in Spark

Caching in Spark refers to storing the results of a Dataframe in memory or on disk of the processing nodes in a cluster.

Caching improves the speed of subsequent transformations or actions as the data likely no longer needs to be retrieved from the original data source. Using caching reduces the resource utilization of the cluster - there is less need to access the storage, networking, and CPU of the Spark nodes as the data is likely already present.

### Disadvantages of caching

- Very large datasets may not fit in memory.
- Local disk based caching may not be a performance improvement.
- Cached objects may not be available.

### Caching tips

- Cache only if you need it
- Try caching Dataframes at various points and determine if your performance improves
- Cache in memory and fast SSD / NVMe storage
- Cache to slow local disk if needed
- Use intermediate files!
- Stop caching objects when finished

### Implementing caching

Call `.cache()` on the Dataframe before action. It can be considered as a transformation, nothing is actually cached until an action is called.

Use `.is_cached` boolean property which returns True if the Dataframe is cached.

To un-cache a Dataframe, we call `.unpersist()`.

In [7]:
# Load the CSV file
df = spark.read.format('csv').options(Header=True).load('datasets/DallasCouncilVoters.csv.gz')
df.cache().count()

44625

In [8]:
df = df.withColumn('ID', F.monotonically_increasing_id())
df = df.cache()
df.show()

+----------+-------------+-------------------+---+
|      DATE|        TITLE|         VOTER_NAME| ID|
+----------+-------------+-------------------+---+
|02/08/2017|Councilmember|  Jennifer S. Gates|  0|
|02/08/2017|Councilmember| Philip T. Kingston|  1|
|02/08/2017|        Mayor|Michael S. Rawlings|  2|
|02/08/2017|Councilmember|       Adam Medrano|  3|
|02/08/2017|Councilmember|       Casey Thomas|  4|
|02/08/2017|Councilmember|Carolyn King Arnold|  5|
|02/08/2017|Councilmember|       Scott Griggs|  6|
|02/08/2017|Councilmember|   B. Adam  McGough|  7|
|02/08/2017|Councilmember|       Lee Kleinman|  8|
|02/08/2017|Councilmember|      Sandy Greyson|  9|
|02/08/2017|Councilmember|  Jennifer S. Gates| 10|
|02/08/2017|Councilmember| Philip T. Kingston| 11|
|02/08/2017|        Mayor|Michael S. Rawlings| 12|
|02/08/2017|Councilmember|       Adam Medrano| 13|
|02/08/2017|Councilmember|       Casey Thomas| 14|
|02/08/2017|Councilmember|Carolyn King Arnold| 15|
|02/08/2017|Councilmember| Rick

In [9]:
df.is_cached

True

In [10]:
df.unpersist()
df.is_cached

False

In [11]:
import time

start_time = time.time()

# Create departures_df
departures_df = spark.read.format('csv').options(Header=True).load('datasets/AA_DFW_2015_Departures_Short.csv.gz')

# Add caching to the unique rows in departures_df
departures_df = departures_df.distinct().cache()

# Count the unique rows in departures_df, noting how long the operation takes
print("Counting %d rows took %f seconds" % (departures_df.count(), time.time() - start_time))

# Count the rows again, noting the variance in time of a cached DataFrame
start_time = time.time()
print("Counting %d rows again took %f seconds" % (departures_df.count(), time.time() - start_time))

Counting 146558 rows took 1.149859 seconds
Counting 146558 rows again took 0.306027 seconds


## 6. Improve import performance

### Spark clusters

Spark Clusters consist of 2 types of processes - one driver process and as many worker processes as required. The driver handles task assignments and consolidation of the data results from the workers. The workers typically handle the actual transformation / action tasks of a Spark job. Once assigned tasks, they operate fairly independently and report results back to the driver. It is possible to have a single node Spark cluster but rare in a production environment.

### Import performance

More smaller objects better than larger ones with the same total amount. Spark can import via wildcard, for example:
```python
airport_df = spark.read.csv('airports-*.txt.gz')
```

We can split objects using OS utilities/scripts (split, cut, awk)
```bash
split -l 10000 -d largefile chunk-
```
or write out to Parquet
```bash
df_csv = spark.read.csv('singlelargefile.csv')
df_csv.write.parquet('data.parquet')
df = spark.read.parquet('data.parquet')
```

Best practice (?): Number of partitions/small files > Number of available nodes.

Well-defined schemas in Spark drastically improve import performance. Without a schema defined, import tasks require reading the data multiple times to infer structure. This is very slow and Spark may not define the objects in the data as expected as well. Spark schemas also provide validation on import, which can save steps with data cleaning jobs and improve the overall processing time.

## 7. Cluster configurations

### Configuration options

Spark has many available configuration settings controlling all aspects of the installation. These configurations can be modified to best match the specific needs for the cluster. The configurations are available in (1) the configuration files, (2) via the Spark web interface, and (3) via the run-time code.

For the third option, to read a configuration setting, call `spark.conf.get()` with the name of the setting as the argument. To write a configuration setting, call `spark.conf.set()` with the name of the setting and the actual value as the function arguments.

### Cluster Types

- Single node clusters: deploy all components on a single system (physical / VM / container).
- Standalone clusters: with dedicated machines as the driver and workers.
- Managed clusters: meaning that the cluster components are handled by a third party cluster manager (YARN, Mesos, Kubernetes).

### Driver

There is one driver per Spark cluster. The driver is responsible for:
- Handling task assignment for nodes/processes in the cluster.
- Monitoring the state of all processes and tasks
- Handling any task retries
- Consolidating results from the other processes in the cluster
- Handling any access to shared data and verifies each worker process has the necessary resources (code, data, etc.)

It is often worth increasing the specifications of the node compared to other systems. Doubling the memory compared to other nodes is recommended. This is useful for task monitoring and data consolidation tasks. As with all Spark systems, fast local storage is useful for running Spark in an ideal setup.

### Worker

A Spark worker handles running tasks assigned by the driver and communicates those results back to the driver.

Ideally, the worker has a copy of all code, data, and access to the necessary resources required to complete a given task. If any of these are unavailable, the worker must pause to obtain the resources.

Recommendations:
- Depending on the type of task, more worker nodes is often better than larger nodes.
- Test various configurations to find the correct balance for your workload.
- Workers can make use of fast local storage (SSD/NVMe) for caching, intermediate files, etc.

In [14]:
# Name of the Spark application instance
app_name = spark.conf.get('spark.app.name')

# Driver TCP port
driver_tcp_port = spark.conf.get('spark.driver.port')

# Number of join partitions
num_partitions = spark.conf.get('spark.sql.shuffle.partitions')

# Show the results
print("Name: %s" % app_name)
print("Driver TCP port: %s" % driver_tcp_port)
print("Number of partitions: %s" % num_partitions)

Name: Spark
Driver TCP port: 60519
Number of partitions: 200


## 8. Performance improvements

### Spark execution plan

To see the estimated plan that will be run to generate results from the DataFrame, we can use `.explain()`.

### Shuffling

Shuffling is the moving of data fragments to various workers as required to complete certain tasks. Shuffling is useful and hides overall complexity from the user. That being said, it can be slow to complete the necessary transfers, especially if a few nodes require all the data. Shuffling lowers the overall throughput of the cluster as the workers must spend time waiting for the data to transfer. This limits the amount of available workers for the remaining tasks in the system. Shuffling is often a necessary component, but it's helpful to try to minimize it as much as possible.

### How to limit shuffling?

Repartitioning by using `.repartition(num_partitions)` requires a full shuffle of data between nodes & processes and is quite costly.

If you need to reduce the number of partitions, use the `.coalesce(num_partitions)` instead. It takes a number of partitions smaller than the current one and consolidates the data without requiring a full data shuffle. (Calling `.coalesce()` with a larger number of partitions does not actually do anything).

Calling `.join()` indiscriminately can often cause shuffle operations, leading to increased cluster load & slower processing times. To avoid some of the shuffle operations whne joining Spark DataFrames, you can use the `.broadcast()` function.

### Broadcasting

Broadcasting in Spark is a method to provide a copy of an object to each worker. When each worker has its own copy of the data, there is less need for communication between nodes. This limits data shuffles and it's more likely a node will fulfil tasks independently.

Using broadcasting can drastically speed up `.join()` operations, especially if one of the Dataframes being joined is much smaller than the other. Broadcasting can slow operations when using very small Dataframes or if you broadcast the larger Dataframe in a join. Spark will often optimize this for you, but as usual, run tests in your environment for best performance.

In [19]:
user_df = spark.read.json('user.json')
label_df = spark.read.csv('label.csv', header=True)

# Join the user_df and label_df
full_df = user_df.join(label_df, user_df['id'] == label_df['id'])

# Show the query plan
full_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [id#922], [id#964], Inner
   :- Sort [id#922 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(id#922, 200), ENSURE_REQUIREMENTS, [plan_id=438]
   :     +- Filter isnotnull(id#922)
   :        +- FileScan json [created_at#919,description#920,entities#921,id#922,location#923,name#924,pinned_tweet_id#925L,profile_image_url#926,protected#927,public_metrics#928,url#929,username#930,verified#931,withheld#932] Batched: false, DataFilters: [isnotnull(id#922)], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/Users/mufin/Downloads/Twibot-22/user.json], PartitionFilters: [], PushedFilters: [IsNotNull(id)], ReadSchema: struct<created_at:string,description:string,entities:struct<description:struct<cashtags:array<str...
   +- Sort [id#964 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(id#964, 200), ENSURE_REQUIREMENTS, [plan_id=439]
         +- Filter isnotnull(id#964)
            +- FileSca

In [20]:
# Import the broadcast method from pyspark.sql.functions
from pyspark.sql.functions import broadcast

# Join the DataFrames using broadcasting
broadcast_df = user_df.join(broadcast(label_df), user_df['id'] == label_df['id'])

# Show the query plan and compare against the original
broadcast_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [id#922], [id#964], Inner, BuildRight, false
   :- Filter isnotnull(id#922)
   :  +- FileScan json [created_at#919,description#920,entities#921,id#922,location#923,name#924,pinned_tweet_id#925L,profile_image_url#926,protected#927,public_metrics#928,url#929,username#930,verified#931,withheld#932] Batched: false, DataFilters: [isnotnull(id#922)], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/Users/mufin/Downloads/Twibot-22/user.json], PartitionFilters: [], PushedFilters: [IsNotNull(id)], ReadSchema: struct<created_at:string,description:string,entities:struct<description:struct<cashtags:array<str...
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, false]),false), [plan_id=464]
      +- Filter isnotnull(id#964)
         +- FileScan csv [id#964,label#965] Batched: false, DataFilters: [isnotnull(id#964)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/mufin/Download

In [22]:
start_time = time.time()
# Count the number of rows in the normal DataFrame
normal_count = full_df.count()
normal_duration = time.time() - start_time

start_time = time.time()
# Count the number of rows in the broadcast DataFrame
broadcast_count = broadcast_df.count()
broadcast_duration = time.time() - start_time

# Print the counts and the duration of the tests
print("Normal count:\t\t%d\tduration: %f" % (normal_count, normal_duration))
print("Broadcast count:\t%d\tduration: %f" % (broadcast_count, broadcast_duration))

Normal count:		1000000	duration: 5.483439
Broadcast count:	1000000	duration: 4.245647
